In [1]:
import pandas as pd

df = pd.read_csv("E:\\1JOB\\Kaiburr\\ML\\complaints.csv\\Complaints_SpacyProcessed.csv", usecols= ["Category","Consumer complaint narrative"])

In [3]:
# import tensorflow as tf
# import tensorflow_hub as hub
# import tensorflow_text as text


# bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_Encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [4]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset

In [6]:
texts = df['Consumer complaint narrative'].tolist()
labels = df['Category'].tolist()

In [7]:
# Splitting the dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)

In [8]:
# Tokenizing the data
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [9]:
# Convert to PyTorch Dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)

In [11]:
# Load DistilBERT for sequence classification with 4 classes
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.

In [13]:
# Training the model
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    save_strategy="epoch",
    report_to="none"  # Change to 'tensorboard' if you want Tensorboard logs
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

c:\Users\sweekar\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/107184 [00:00<?, ?it/s]

{'loss': 1.0173, 'learning_rate': 4.99533512464547e-05, 'epoch': 0.0}
{'loss': 0.7226, 'learning_rate': 4.990670249290939e-05, 'epoch': 0.01}
{'loss': 0.6261, 'learning_rate': 4.986005373936409e-05, 'epoch': 0.01}
{'loss': 0.5854, 'learning_rate': 4.981340498581878e-05, 'epoch': 0.01}


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 100663296 bytes.

In [ ]:
# Evaluating the model
predictions = trainer.predict(test_dataset).predictions.argmax(axis=-1)

print(classification_report(test_labels, predictions))